# Orchestration Workflow

## Install Library

In [ ]:
%pip install azure-ai-language-conversations

## Load Azure Configurations

In [11]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

# Represents the Project Name
project_name = "Python-Orchestration-Project"
# Represents the output model label.
modelLabel = "Python-Orchestration-Model"  

## Create ConversationAuthoringClient

In [12]:
from azure.ai.language.conversations.authoring import ConversationAuthoringClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure ConversationAuthoringClient using the provided key and endpoint.

    Returns:
        ConversationAuthoringClient: An authenticated client for Azure Conversation Authoring.
    """
    credential = AzureKeyCredential(language_key)
    client = ConversationAuthoringClient(
        endpoint=language_endpoint,
        credential=credential
    )
    return client

# Initialize the client
conversation_authoring_client = authenticate_client()

## Create a Project

In [13]:
project = {
        "language": "en-us",  # The project language. This is BCP-47 representation of a language. For example, use "en" for English, "en-gb" for English (UK), "es" for Spanish etc. Required.
        "projectKind": "Orchestration",  # Represents the project kind. Required. Known values are: "Conversation" and "Orchestration".
        "projectName": project_name,  # The new project name. Required.
        "description": "sample orchestration workflow made in Python",  # Optional. The project description.
        "multilingual": False,  # Optional. Whether the project would be used for multiple languages or not.
        "settings": {
           "confidenceThreshold": 0.0,  # The threshold of the intent with the highest confidence, at which the prediction will automatically be changed to None. The value of the threshold should be between 0 and 1 inclusive. Required.
       }
   }

conversation_authoring_client.create_project(project_name, project)

{'createdDateTime': '0001-01-01T00:00:00Z',
 'lastModifiedDateTime': '0001-01-01T00:00:00Z',
 'projectKind': 'Orchestration',
 'settings': {'confidenceThreshold': 0.0,
  'normalizeCasing': False,
  'augmentDiacritics': False},
 'projectName': 'Python-Orchestration-Project',
 'multilingual': False,
 'description': 'sample orchestration workflow made in Python',
 'language': 'en-us'}

## Define Project Assets and Intents

In [14]:
exported_project_assets = {
    "projectKind": "Orchestration",
    "intents": [
        {
            "category": "None"
        },
        {
            "category": "Intent_CLU",
            "orchestration": {
                "targetProjectKind": "Conversation",
                "conversationOrchestration": {
                    "projectName": "MyProject1",
                    "deploymentName": "RestaurantCLU"
                }
            }
        },
        {
            "category": "Intent_QA",
            "orchestration": {
                "targetProjectKind": "QuestionAnswering",
                "questionAnsweringOrchestration": {
                    "projectName": "SurfaceProject"
                }
            }
        }
    ],
    "entities": [],
    "utterances": []
}

## Import Project

In [15]:
poller = conversation_authoring_client.begin_import_project(
    project_name=project_name,
    project={
        "assets": exported_project_assets,
        "metadata": {
            "projectKind": "Orchestration",
            "settings": {
                "confidenceThreshold": 0.0,
            },
            "projectName": project_name,
            "multilingual": False,
            "language": "en-us",
        },
        "projectFileVersion": "2022-05-01",
        "stringIndexType": "Utf16CodeUnit"  # Add this line to specify the StringIndexType
    },
)
response = poller.result()
print(response)

{'jobId': 'afb63886-04af-4325-9c7a-9be493424cee_638802720000000000', 'createdDateTime': '2025-04-15T09:32:46Z', 'lastUpdatedDateTime': '2025-04-15T09:32:46Z', 'expirationDateTime': '2025-04-22T09:32:46Z', 'status': 'succeeded'}


## Train Model

In [17]:
# JSON input template you can fill out and use as your body input.
configuration = {
    "modelLabel": modelLabel,  # Represents the output model label. Required.
    "trainingMode": "standard",  # Represents the mode of the training operation. Required. Known values are: "advanced" and "standard".
    "evaluationOptions": {
        "kind": "percentage",  # Optional. Represents the evaluation kind. By default, the evaluation kind is set to percentage. Known values are: "percentage" and "manual".
        "testingSplitPercentage": 20,  # Optional. Represents the testing dataset split percentage. Only needed in case the evaluation kind is percentage.
        "trainingSplitPercentage": 80  # Optional. Represents the training dataset split percentage. Only needed in case the evaluation kind is percentage.
    },
    #"trainingConfigVersion": "str"  # Optional. Represents training config version. By default, "latest" value is used which uses the latest released training config version.
   }

poller = conversation_authoring_client.begin_train(project_name=project_name,configuration=configuration)
response = poller.result()
print(response)

{'result': {'modelLabel': 'Python-Orchestration-Model', 'trainingConfigVersion': '2022-05-01', 'trainingMode': 'standard', 'trainingStatus': {'percentComplete': 100, 'startDateTime': '2025-04-15T09:34:09.6724141Z', 'endDateTime': '2025-04-15T09:34:13.2471617Z', 'status': 'succeeded'}, 'evaluationStatus': {'percentComplete': 100, 'startDateTime': '2025-04-15T09:34:16.3562974Z', 'endDateTime': '2025-04-15T09:34:32.5263075Z', 'status': 'succeeded'}}, 'jobId': 'f2e682b6-3df5-490f-932b-957fabaeb719_638802720000000000', 'createdDateTime': '2025-04-15T09:34:08Z', 'lastUpdatedDateTime': '2025-04-15T09:35:16Z', 'expirationDateTime': '2025-04-22T09:34:08Z', 'status': 'succeeded', 'warnings': []}


## Display Model Evaluation Summary

In [18]:
evaluation_summary = conversation_authoring_client.get_model_evaluation_summary(project_name=project_name, trained_model_label=modelLabel)
print(evaluation_summary)

{'intentsEvaluation': {'confusionMatrix': {'None': {}, 'Intent_CLU': {'Intent_CLU': {'normalizedValue': 100.0, 'rawValue': 9.0}}, 'Intent_QA': {'Intent_QA': {'normalizedValue': 96.666664, 'rawValue': 29.0}, 'Intent_CLU': {'normalizedValue': 3.3333333, 'rawValue': 1.0}}}, 'intents': {'Intent_QA': {'f1': 0.9830507636070251, 'precision': 1.0, 'recall': 0.9666666388511658, 'truePositiveCount': 29, 'trueNegativeCount': 9, 'falsePositiveCount': 0, 'falseNegativeCount': 1}, 'Intent_CLU': {'f1': 0.9473683834075928, 'precision': 0.8999999761581421, 'recall': 1.0, 'truePositiveCount': 9, 'trueNegativeCount': 29, 'falsePositiveCount': 1, 'falseNegativeCount': 0}, 'None': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'truePositiveCount': 0, 'trueNegativeCount': 0, 'falsePositiveCount': 0, 'falseNegativeCount': 0}}, 'microF1': 0.974359, 'microPrecision': 0.974359, 'microRecall': 0.974359, 'macroF1': 0.9652096, 'macroPrecision': 0.95, 'macroRecall': 0.98333335}, 'evaluationOptions': {'kind': 'percent

## Deploy the Trained Model

In [19]:
deployment = {
       "trainedModelLabel": modelLabel  # Represents the trained model label. Required.
   }

trained_model = conversation_authoring_client.begin_deploy_project(project_name=project_name, deployment_name=modelLabel, deployment=deployment)
response = trained_model.result()
print(response)

{'deploymentName': 'Python-Orchestration-Model', 'modelId': 'Python-Orchestration-Model-20250415T093516-628b10700141472cbf6ff157e36073c2', 'lastTrainedDateTime': '2025-04-15T09:35:16.6279822Z', 'lastDeployedDateTime': '2025-04-15T09:37:07Z', 'deploymentExpirationDate': '2026-10-27', 'modelTrainingConfigVersion': '2022-05-01'}


## Making the Request


## Create a ConversationAnalysisClient

In [20]:
from azure.ai.language.conversations import ConversationAnalysisClient
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(language_key)
conversation_analysis_client = ConversationAnalysisClient(
        endpoint=language_endpoint,
        credential=credential
)

## Helper function to Send requests

In [58]:
# Analyze the utterance using the deployed model
def analyze_utterance(utterance):
    """
    Analyzes the given utterance using the deployed model.

    Args:
        utterance (str): The utterance to analyze.

    Returns:
        None
    """
    result = conversation_analysis_client.analyze_conversation(
        task={
            "kind": "Conversation",
            "analysisInput": {
                "conversationItem": {
                    "participantId": "1",
                    "id": "1",
                    "modality": "text",
                    "language": "en",
                    "text": utterance
                },
                "isLoggingEnabled": False
            },
            "parameters": {
                "projectName": project_name,
                "deploymentName": modelLabel,
                "verbose": True
            }
        }
    )

    if result.get('kind') == 'ConversationResult':
        prediction = result['result']['prediction']
        print(f"Query: {result['result']['query']}")
        print(f"Top Intent: {prediction['topIntent']}")
        print(f"Project Kind: {prediction['projectKind']}")
        print("\nIntents:")
        for intent, details in prediction['intents'].items():
            print(f"  - {intent}:")
            print(f"    Confidence Score: {details['confidenceScore']}")
            if 'result' in details:
                print(f"    Sub-Intent: {details['result']['prediction']['topIntent']}")
                print("    Sub-Intent Confidence Scores:")
                for sub_intent in details['result']['prediction']['intents']:
                    print(f"      - {sub_intent['category']}: {sub_intent['confidenceScore']}")
        print("\nEntities:")
        for entity in prediction['intents']['Intent_CLU']['result']['prediction']['entities']:
            print(f"  - {entity['category']}: {entity['text']}")
            print(f"    Confidence Score: {entity['confidenceScore']}")
            print(f"    Resolutions: {entity['resolutions']}")
            print(f"    Extra Information: {entity['extraInformation']}")
    else:
        print("Invalid result format.")


In [59]:
# Sample utterance to analyze
utterance = "I'd like to book a table for 4 people at 7 PM on April 20th."
analyze_utterance(utterance)

Query: I'd like to book a table for 4 people at 7 PM on April 20th.
Top Intent: Intent_CLU
Project Kind: Orchestration

Intents:
  - Intent_CLU:
    Confidence Score: 0.95507115
    Sub-Intent: BookTable
    Sub-Intent Confidence Scores:
      - BookTable: 0.9831086
      - OrderFood: 0.8924247
      - CheckOpeningHours: 0.7405365
      - None: 0
  - Intent_QA:
    Confidence Score: 0.3024424
  - None:
    Confidence Score: 0

Entities:
  - Number: 4
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'NumberResolution', 'numberKind': 'Integer', 'value': 4}]
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'quantity.number'}]
  - Number: 7
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'NumberResolution', 'numberKind': 'Integer', 'value': 7}]
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'quantity.number'}]
  - Date: 7 PM on April 20th
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'DateTimeReso

In [60]:
# Sample utterance to analyze
utterance = "How do you setup Surface?"
analyze_utterance(utterance)

Query: How do you setup Surface?
Top Intent: Intent_QA
Project Kind: Orchestration

Intents:
  - Intent_QA:
    Confidence Score: 0.91513956


KeyError: 'prediction'

## Export project to save it

In [ ]:
export_result = conversation_authoring_client.begin_export_project(
    project_name=project_name, 
    string_index_type="Utf16CodeUnit",
    trained_model_label=modelLabel,
    
).result()

print(export_result)

{'resultUrl': 'https://ziggylanguagedemocomplete.cognitiveservices.azure.com/language/authoring/analyze-conversations/projects/CLU-Sample-Python/export/jobs/a8c38517-b7f5-4d7e-892c-dc6c58299c16_638802720000000000/result?api-version=2023-04-01', 'jobId': 'a8c38517-b7f5-4d7e-892c-dc6c58299c16_638802720000000000', 'createdDateTime': '2025-04-15T03:23:50Z', 'lastUpdatedDateTime': '2025-04-15T03:23:50Z', 'expirationDateTime': '2025-04-22T03:23:50Z', 'status': 'succeeded'}


## Save data into a JSON File

In [34]:
import requests
import json
import os

# URL from the result
result_url = export_result['resultUrl']

# Azure API Key (replace with your actual key)
api_key = os.environ.get("AZURE_AI_LANGUAGE_KEY")

# Make the GET request
headers = {
    "Ocp-Apim-Subscription-Key": api_key
}
response = requests.get(result_url, headers=headers)

# Check the response status
if response.status_code == 200:
    result_data = response.json()
    
    # Print the result_data in a properly formatted JSON
    print(json.dumps(result_data, indent=4))
    
    # Optionally, save it to a file
    with open("exported_project_data.json", "w") as json_file:
        json.dump(result_data, json_file, indent=4)
else:
    print(f"Failed to fetch data. Status Code: {response.status_code}, Response: {response.text}")

{
    "projectFileVersion": "2023-04-01",
    "stringIndexType": "Utf16CodeUnit",
    "metadata": {
        "projectKind": "Conversation",
        "settings": {
            "confidenceThreshold": 0.0,
            "normalizeCasing": false,
            "augmentDiacritics": false
        },
        "projectName": "CLU-Sample-Python",
        "multilingual": false,
        "description": "Sample project made in Python",
        "language": "en"
    },
    "assets": {
        "projectKind": "Conversation",
        "intents": [
            {
                "category": "BookTable"
            },
            {
                "category": "OrderFood"
            },
            {
                "category": "None"
            }
        ],
        "entities": [
            {
                "category": "Date",
                "compositionSetting": "combineComponents",
                "prebuilts": [
                    {
                        "category": "DateTime"
                    }
       

## Import Data using JSON file

In [35]:
import json

# Filepath to the JSON file
file = "exported_project_data.json"

# Open and load the JSON file
with open(file, "r") as json_file:
    data = json.load(json_file)

# Import the project
poller = conversation_authoring_client.begin_import_project(
    project_name="Imported_Project_Sample",
    project=data,
)
response = poller.result()
print(response)

{'jobId': '32f1d127-0f41-45bc-9242-786ea936ca77_638802720000000000', 'createdDateTime': '2025-04-15T03:24:04Z', 'lastUpdatedDateTime': '2025-04-15T03:24:04Z', 'expirationDateTime': '2025-04-22T03:24:04Z', 'status': 'succeeded'}
